In [ ]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from dotenv import load_dotenv

from app.telegram_client import get_client_with_auth
from app.chroma_client import get_embeddings, get_client, load_data_to_chroma


load_dotenv()

CHANNEL_NAME = os.getenv('TELEGRAM_CHAT_NAME')
CHANNEL_ID = os.getenv('TELEGRAM_CHAT_ID')

try:
    CHANNEL_ID = int(CHANNEL_ID)
except:
    pass

print(f"Channel id: {CHANNEL_ID}")

In [ ]:
telegram_client = await get_client_with_auth()

In [ ]:
from typing import Any
from tqdm import tqdm
from telethon.tl.types import PeerChannel

CHAT_MESSAGES_LIMIT = 1000

async def get_messages_from_channel(
    telegram_client, channel_id: int | str, min_id: int = 0, max_id: int = 0, chunks: int = 10
) -> dict[int, dict[str, Any]]:
    messages_data = {}
    if isinstance(channel_id, int) and channel_id < 0:
        channel_id = PeerChannel(channel_id)

    channel = await telegram_client.get_entity(channel_id)
    last_seen_message_id = max_id

    for _i in tqdm(range(chunks), desc="Fetching chunks", position=0, leave=False, colour='green'):
        if all((last_seen_message_id, min_id)) and last_seen_message_id == min_id:
            print("Reached last unseen message")
            break

        async for message in telegram_client.iter_messages(
            channel, min_id=min_id, offset_id=last_seen_message_id, limit=CHAT_MESSAGES_LIMIT
        ):
            if not message.text or len(message.text) < 10:
                continue

            message_text = message.text
            if message.is_reply:
                if message.reply_to_msg_id not in messages_data:
                    original_message = await message.get_reply_message()
                    if original_message:
                        messages_data[original_message.id] = {
                            'text': original_message.text,
                            'metadata': {
                                'sender_name': original_message.sender.username or '',
                                'id': original_message.id,
                                'date_str': original_message.date.isoformat(),
                                'date': original_message.date.timestamp(),
                            }
                        }

                original_message = messages_data.get(message.reply_to_msg_id)
                if original_message:
                    message_text = f'>> {original_message['text']}\n\n {message.text}'
                else:
                    message_text = f'>> [[ORIGINAL MESSAGE REMOVED]]\n\n {message.text}'

            messages_data[message.id] = {
                'text': message_text,
                'metadata': {
                    'sender_name': message.sender.username or '',
                    'id': message.id,
                    'date': message.date.timestamp(),
                }
            }

        last_seen_message_id = message.id
        if last_seen_message_id == 0:
            break

    return messages_data

In [ ]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{CHANNEL_NAME}', embeddings)

In [ ]:
from langchain_chroma import Chroma
from datetime import datetime, timedelta, UTC

def get_last_seen_id_from_db(chroma_client: Chroma):
    doc_dict = chroma_client.get(
        where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
        include=['metadatas'],
    )
    if not doc_dict['metadatas']:
        return 0

    return max(meta['id'] for meta in doc_dict['metadatas'])

def get_earliest_message_id(chroma_client: Chroma):
    doc_dict = chroma_client.get(include=['metadatas'])
    if not doc_dict['metadatas']:
        return 0

    return min(meta['id'] for meta in doc_dict['metadatas'])

async def get_early_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    earliest_message_id = get_earliest_message_id(chroma_client)
    print("Earliest seen id:", earliest_message_id)
    messages = await get_messages_from_channel(
        telegram_client, channel_id, max_id=earliest_message_id, chunks=chunks
    )
    print("Got messages:", len(messages))
    return messages

async def get_last_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    last_seen_id = get_last_seen_id_from_db(chroma_client)
    print("Last seen id:", last_seen_id)
    messages = await get_messages_from_channel(telegram_client, channel_id, min_id=last_seen_id, chunks=chunks)
    print("Got messages:", len(messages))
    return messages

In [ ]:
new_messages = await get_last_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

In [ ]:
new_messages = await get_early_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

In [ ]:
print(len(new_messages))
print(list(new_messages.values())[-1:])

In [ ]:
load_data_to_chroma(chroma_client_from_telegram, new_messages.values(), reset=True)

In [ ]:
from langchain import PromptTemplate
from langchain_ollama import ChatOllama


llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [ ]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения в чате."
user_query = "Можно ли компенсировать отель или аквапарк через велнес БТА или BTA?"
user_query = "Как получить детские деньги, компенсацию по уходу за ребенком."
user_query = "Тенденции рынка недвижимости на Кипре"

In [ ]:
ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
    Напиши {n} гипотетических ответов на запрос пользователя.
    Не добавляй заголовков, авторов и другой дополнительной информации к твоим ответам.

    Вопрос пользователя:
    {user_query}

    Гипотетические ответы:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt_expanding | llm_qwen3_8b
raw_hypothetical_answers = hyde_chain_r1_8b.invoke({"user_query": user_query, "n": 15}).content
raw_hypothetical_answers

In [ ]:
user_query = raw_hypothetical_answers.split('</think>', maxsplit=1)[-1]

In [ ]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(user_query, k=15)
related_docs

In [ ]:

from datetime import datetime, timedelta, UTC

related_docs = chroma_client_from_telegram.get(
    where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
)
meta_docs = list(zip(related_docs['metadatas'], related_docs['documents']))

context = "\n\n---\n\n".join(f"{meta['date']} - {meta['sender_name']}: {doc}" for meta, doc in meta_docs)

In [ ]:
len(related_docs['metadatas'])

In [ ]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{doc.metadata['date']} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))

In [ ]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{doc.metadata['date']} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [ ]:
# user_query = "Здесь собраны последние сообщения из чата. Расскажи про основные темы и выводы."

ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)